In [29]:
import pandas as pd

In [30]:
import pandas as pd
import glob
import os

# Define the path to the CSV files
csv_folder = '../../data/raw/'
csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))

In [31]:
csv_files

['../../data/raw/2019_03_Marc_BICING_US.csv',
 '../../data/raw/2018_11_Novembre_BICING_US.csv',
 '../../data/raw/2018_12_Desembre_BICING_US.csv',
 '../../data/raw/2018_09_Setembre_BICING_US.csv',
 '../../data/raw/2018_08_Agost_BICING_US.csv',
 '../../data/raw/2019_01_Gener_BICING_US.csv',
 '../../data/raw/2019_02_Febrer_BICING_US.csv',
 '../../data/raw/2018_10_Octubre_BICING_US.csv']

In [32]:


# Read and concatenate all CSV files
df_all = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Display the shape or head to verify
print(f"Total rows loaded: {df_all.shape[0]}")
df_all.head()


Total rows loaded: 47270


,error,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage,dateTime
0,0,47,4,43,2019-02-28 23:59:57
1,0,46,3,43,2019-03-01 00:04:57
2,0,40,2,38,2019-03-01 00:09:57
3,0,39,4,35,2019-03-01 00:14:58
4,0,39,5,34,2019-03-01 00:19:57


In [33]:
raw_data = df_all[["bikesInUsage", "electricalBikesInUsage", "mechanicalBikesInUsage", "dateTime"]]

In [34]:
raw_data

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage,dateTime
0,47,4,43,2019-02-28 23:59:57
1,46,3,43,2019-03-01 00:04:57
2,40,2,38,2019-03-01 00:09:57
3,39,4,35,2019-03-01 00:14:58
4,39,5,34,2019-03-01 00:19:57
...,...,...,...,...
47265,192,2,190,2018-10-31 23:36:38
47266,195,2,193,2018-10-31 23:41:38
47267,156,1,155,2018-10-31 23:46:37
47268,139,0,139,2018-10-31 23:51:37


In [35]:
# Convert 'dateTime' to datetime objects
raw_data['dateTime'] = pd.to_datetime(raw_data['dateTime'])

# Round each timestamp to the nearest 15-minute interval
raw_data['dateTime'] = raw_data['dateTime'].dt.round('15min')

# Group by the rounded timestamp and aggregate (mean is typical for continuous variables)
raw_data_15min = raw_data.groupby('dateTime').agg({
    'bikesInUsage': 'mean',
    'electricalBikesInUsage': 'mean',
    'mechanicalBikesInUsage': 'mean'
}).reset_index()

# Set datetime as index for resampling
raw_data_15min.set_index('dateTime', inplace=True)

# Create a continuous 15-minute time series (this will add missing intervals as NaN)
ts_data_15min = raw_data_15min.resample('15min').asfreq()

# Optionally fill missing data
# ts_data_15min = ts_data_15min.interpolate(method='time')  # or use .ffill() / .bfill()

# Inspect result
ts_data_15min

/tmp/ipykernel_45858/4087172884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['dateTime'] = pd.to_datetime(raw_data['dateTime'])
/tmp/ipykernel_45858/4087172884.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['dateTime'] = raw_data['dateTime'].dt.round('15min')


,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage
dateTime,,,
2018-08-01 17:45:00,662.500000,15.000000,647.5
2018-08-01 18:00:00,648.000000,11.000000,637.0
2018-08-01 18:15:00,845.000000,13.000000,832.0
2018-08-01 18:30:00,889.000000,13.000000,876.0
2018-08-01 18:45:00,817.000000,12.000000,805.0
...,...,...,...
2019-03-24 23:00:00,2.000000,2.000000,0.0
2019-03-24 23:15:00,3.000000,3.000000,0.0
2019-03-24 23:30:00,2.333333,2.333333,0.0


In [36]:
ts_data_15min.isna().sum()


bikesInUsage              2399
electricalBikesInUsage    2399
mechanicalBikesInUsage    2399
dtype: int64

In [37]:
ts_data_15min.loc['2019-01-19 23:00:00':'2019-01-20 23:59:59'].head(10)

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage
dateTime,,,
2019-01-19 23:00:00,125.000000,0.000000,125.000000
2019-01-19 23:15:00,128.000000,0.333333,127.666667
2019-01-19 23:30:00,122.000000,1.666667,120.333333
2019-01-19 23:45:00,142.666667,1.000000,141.666667
2019-01-20 00:00:00,143.333333,1.333333,142.000000
2019-01-20 00:15:00,164.333333,4.000000,160.333333
2019-01-20 00:30:00,168.333333,4.666667,163.666667
2019-01-20 00:45:00,148.000000,2.333333,145.666667
2019-01-20 01:00:00,133.333333,2.000000,131.333333


In [38]:
ts_data_15min.groupby([ts_data_15min.index.date]).sum()

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage
2018-08-01,14781.500000,160.000000,14621.5
2018-08-02,36273.000000,397.000000,35876.0
2018-08-03,33880.000000,359.000000,33521.0
2018-08-04,23628.000000,188.000000,23440.0
2018-08-05,21300.000000,236.000000,21064.0
...,...,...,...
2019-03-21,3.000000,2.000000,1.0
2019-03-22,0.000000,0.000000,0.0
2019-03-23,0.000000,0.000000,0.0
2019-03-24,284.666667,284.666667,0.0


In [39]:
ts_data_15min.loc['2018-09-27 00:00:00':'2018-09-27 23:59:59']

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage
dateTime,,,
2018-09-27 00:00:00,NaN,NaN,NaN
2018-09-27 00:15:00,NaN,NaN,NaN
2018-09-27 00:30:00,NaN,NaN,NaN
2018-09-27 00:45:00,NaN,NaN,NaN
2018-09-27 01:00:00,NaN,NaN,NaN
...,...,...,...
2018-09-27 22:45:00,NaN,NaN,NaN
2018-09-27 23:00:00,NaN,NaN,NaN
2018-09-27 23:15:00,NaN,NaN,NaN


In [40]:
# Ensure the index is datetime
ts_data_15min.index = pd.to_datetime(ts_data_15min.index)

# Create a DataFrame to hold the summary
summary = ts_data_15min.copy()
summary['month'] = summary.index.month
summary['year'] = summary.index.year
summary['day'] = summary.index.date

# Group by year and month to get:
# - number of unique days (i.e., days with at least one time point)
# - total number of rows
monthly_summary = summary.groupby(['year', 'month']).agg(
    days_in_month=('day', lambda x: len(set(x))),  # unique days
    total_rows=('day', 'count')  # number of 15-min records
).reset_index()

# Optional: Add total across all months
total_days = monthly_summary['days_in_month'].sum()
total_rows = monthly_summary['total_rows'].sum()

# Append total row
total_row = pd.DataFrame([{
    'year': '',
    'month': '',
    'days_in_month': total_days,
    'total_rows': total_rows
}])
monthly_summary = pd.concat([monthly_summary, total_row], ignore_index=True)

# Display result
monthly_summary

,year,month,days_in_month,total_rows
0,2018,8,31,2905
1,2018,9,30,2880
2,2018,10,31,2976
3,2018,11,30,2880
4,2018,12,31,2976
5,2019,1,31,2976
6,2019,2,28,2688
7,2019,3,25,2305
8,,,237,22586


In [41]:
# Reindex to ensure complete 15-minute grid
full_index = pd.date_range(start='2018-08-01', end='2019-03-25 23:45:00', freq='15min')
ts_data_15min = ts_data_15min.reindex(full_index)

# Name the index (since reindexing can drop the name)
ts_data_15min.index.name = 'dateTime'
ts_data_15min

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage
dateTime,,,
2018-08-01 00:00:00,NaN,NaN,NaN
2018-08-01 00:15:00,NaN,NaN,NaN
2018-08-01 00:30:00,NaN,NaN,NaN
2018-08-01 00:45:00,NaN,NaN,NaN
2018-08-01 01:00:00,NaN,NaN,NaN
...,...,...,...
2019-03-25 22:45:00,NaN,NaN,NaN
2019-03-25 23:00:00,NaN,NaN,NaN
2019-03-25 23:15:00,NaN,NaN,NaN


In [42]:
ts_data_15min.loc['2018-09-23 00:00:00':'2018-09-23 23:59:59']

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage
dateTime,,,
2018-09-23 00:00:00,380.0,8.0,372.0
2018-09-23 00:15:00,362.0,4.0,358.0
2018-09-23 00:30:00,352.0,6.0,346.0
2018-09-23 00:45:00,307.0,5.0,302.0
2018-09-23 01:00:00,327.0,5.0,322.0
...,...,...,...
2018-09-23 22:45:00,353.0,4.0,349.0
2018-09-23 23:00:00,409.0,5.0,404.0
2018-09-23 23:15:00,407.0,6.0,401.0


In [43]:
ts_data_15min = ts_data_15min.round(2)


In [44]:
ts_data_15min.to_csv("ts_data_15min.csv")